In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statistics import mean
import matplotlib.font_manager

from RoomMeetingAssignments import *
from PeerConnection import *
from dt_calc_models import *

from rmsops import *

fontName = 'YS Text'
fontColor = '#FF6427'
fontColor2 = '#000000'

font_files = matplotlib.font_manager.findSystemFonts(fontpaths='/usr/share/fonts/truetype')
for font_file in font_files:
    try: 
        matplotlib.font_manager.fontManager.addfont(font_file)
    except Exception:
        print('failed to add font ' + font_file)
        #ignore
font = {'family' : 'YS Text',
        'size'   : 16}

matplotlib.rc('font', **font)

In [ ]:
#to speed up testing
MAX_NUM_ROWS=50000
# MAX_NUM_ROWS=10000000000

MEDIASERVER_IDLE_TIMEOUT = 60

#model parameters
ISLAND_SIZES = [50] * 5
RECONNECT_DOWNTIME_SECONDS = 10

#parameters to optimize
GRACE_PERIOD_SECONDS = 1800

#disruptions are done one island at a time. When one rolls out, other islands stay 100% available
DISRUPTION_BUDGET = 50

In [ ]:
def plotResult(sourceFName):
    downtimesToPlot = RMSDowntimeChart(sourceFname=sourceFName)

    print(f"{sourceFName}\ntotal downtime: {downtimesToPlot.totalDT}")
    
    fig = plt.figure(figsize=(16, 3))
    ax = fig.add_subplot(111)
    ax.set_title('Downtime Derivative', fontsize=36, fontname=fontName)
    ax.set_xlabel('DateTime', fontsize=22, fontname=fontName)
    ax.set_ylabel('Downtime', fontsize=22, fontname=fontName)
    plt.scatter(x=downtimesToPlot.datetimes, y=downtimesToPlot.dts, color=fontColor, label='Downtimes')
    ax.grid()
    plt.legend(fontsize=22, loc ="upper right")
    
    fig = plt.figure(figsize=(16, 3))
    ax = fig.add_subplot(111)
    ax.set_title('Interrupted connections', fontsize=36, fontname=fontName)
    ax.set_xlabel('DateTime', fontsize=22, fontname=fontName)
    ax.set_ylabel('PC Interrupted', fontsize=22, fontname=fontName)
    matplotlib.rcParams['lines.markersize']=10
    plt.scatter(x=downtimesToPlot.datetimes, y=downtimesToPlot.pcInterrupted, color=fontColor, label='PC Interrupted')
    matplotlib.rcParams['lines.markersize']=8
    plt.scatter(x=downtimesToPlot.datetimes, y=downtimesToPlot.rmInterrupted, color='Black', label='RM Interrupted')
    ax.grid()
    plt.legend(fontsize=22, loc ="upper right")
    
    fig = plt.figure(figsize=(16, 3))
    ax = fig.add_subplot(111)
    ax.set_title('Total connections', fontsize=36, fontname=fontName)
    ax.set_xlabel('DateTime', fontsize=22, fontname=fontName)
    ax.set_ylabel('RMInterrupted', fontsize=22, fontname=fontName)
    matplotlib.rcParams['lines.markersize']=10
    plt.scatter(x=downtimesToPlot.datetimes, y=downtimesToPlot.pcTotal, color=fontColor, label='PC total')
    matplotlib.rcParams['lines.markersize']=8
    plt.scatter(x=downtimesToPlot.datetimes, y=downtimesToPlot.rmTotal, color='Black', label='RM Total')
    ax.grid()
    plt.legend(fontsize=22, loc ="upper right")

In [ ]:
plotResult('result_RandomIslandLeastLoadedNewNodePolicy.dtmodel_IntegratingDTClacModel.grace_60.disr_3.at_2023-10-02T13:00:00.tsv')